In [ ]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
from sklearn.mixture import GaussianMixture
import glob
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
%matplotlib inline

In [ ]:
#Define List of Users and define File-Path to Users Folder
list_users = ['ana_brandine', 'vicky_regouli', 'luismiguelpss', 'ilariabiagini', 
              'emnegg', 'kerendhahn', 'agiorgina', 'roulamatta', 'jussbieber9827', 
              'eremiaheidr', 'eunhuiheo', 'anastasiakaps', 'achaelilsone', 'orit_talbi',
              'sorayaalassmi', 'altonolnlis', 'vaso1977', 'theunrealobserver', 'nsb.koc',
              'vivpeng', 'amrynevillek', 'danalev7', 'irienyree', 'lilachturgeman', 
              'emel_karakoc', 'thiswhomustbekept', 'j_f_lil', 'ulietteearneye', 
              'gilanaz', 'sarrahdolly', 'alexchahine97', 'photographerarson', 
              'angecanindo', 'fiona_smithson', 'chelsea_xu620']

path = "/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/Datasets/sample_users_media/"
dest_path = "/Users/kmotwani/Desktop/Me - Local/Education/Courses/Capstone Project/Clustering_kNN_"

In [ ]:
#Helper Function to get images from path
def get_images(path, list_users):
    #Get all Images of Users in the List
    user_imgs = {}
    for i in list_users:
        temp_path = path + i
        for j in glob.glob(temp_path + '/*.jpg'):
            file_name = j.replace(temp_path,'')[1:]
            img = io.imread(j)
            user_imgs[(i,file_name)] = img
    return user_imgs 


#Get Images from User List and Path
user_imgs = get_images(path, list_users)
print("Number of images loaded:", len(user_imgs))

In [ ]:
#Helper function to convert image to d-dimension vector
def convert(user_imgs):
    user_features = []
    for i in user_imgs.items():
        r_mean, r_std, r_med = np.mean(i[1][:,:,0].ravel()), np.std(i[1][:,:,0].ravel()), np.median(i[1][:,:,0].ravel())
        g_mean, g_std, g_med  = np.mean(i[1][:,:,1].ravel()), np.std(i[1][:,:,1].ravel()), np.median(i[1][:,:,1].ravel())
        b_mean, b_std, b_med  = np.mean(i[1][:,:,2].ravel()), np.std(i[1][:,:,2].ravel()), np.median(i[1][:,:,2].ravel())
        canny = np.mean(np.ravel(cv2.Canny(cv2.cvtColor(i[1], cv2.COLOR_BGR2HSV),100,200,L2gradient = True)))
        try:
            orb = cv2.ORB_create(100)
            kp = orb.detect(i[1],None)
            kp, des = orb.compute(i[1], kp)
            orb_centers = list(KMeans(1).fit([i.pt for i in kp]).cluster_centers_)
            orbx1, orby1 = orb_centers[0][0]*255/np.shape(i[1])[0], orb_centers[0][1]*255/np.shape(i[1])[1]
        except ValueError:
            continue
        user_features.append(np.array([i[0][0],i[0][1], r_mean, r_std, r_med, g_mean, g_std, g_med, b_mean, b_std, b_med, canny, orbx1, orby1]))
    return user_features
    
#Convert Images
user_features = convert(user_imgs)
print("Feature Vectors Created.")

In [ ]:
#Create Dataframe of Features
df = pd.DataFrame(user_features)
df.columns = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", "G_MED", "B_Mean", "B_STD", "B_MED", "Canny", "ORB_X", "ORB_Y"]
display(df.head())

In [ ]:
#Define Cluster Count
cluster_count = 4

#Define Dataset
data = df.copy(deep=True)
del data["User_Handle"]
del data["URL"]
display(data.head())

#Implement Gaussian Mixture Model Algortihm 
model_gaussian = GaussianMixture(n_components=cluster_count, random_state=9001)

#Fit Model and Predict
model_gaussian.fit(data)
y_pred = model_gaussian.predict_proba(data)

#Add prediction to dataframe     
for i in range(0,cluster_count+1):
    if i==cluster_count:
        label = "Prediction"
        df[label] = model_gaussian.predict(data)
        break
    label = "Prob_" + str(i)
    df[label] = y_pred[:,i]
    

display(df.head())

In [ ]:
#Helper function to create folders for Image Clustering
def save_clusters(df, user_imgs, dest_path, label):
    dest_path += label + "/"
    for i in user_imgs.items():
        temp_row = df[df["URL"]==i[0][1]]
        try:
            name, pred_folder = i[0][1], str(temp_row['Prediction'].values[0])
        except IndexError:
            continue
        temp_path = dest_path + "Cluster" + pred_folder + "/"
        if not os.path.exists(temp_path):
            os.makedirs(temp_path)
        io.imsave(temp_path+name, i[1])

save_clusters(df, user_imgs, dest_path, "Competitors")
print("All Images Saved.")

In [ ]:
#Helper function to obtain percentage of Cluster Presence
def cluster_presence(df):
    cluster_presence = []
    for i in list(df['User_Handle'].unique()):
        user_dict = {}
        temp_df = df[df['User_Handle']==i]
        post_count  =  len(temp_df)
        user_dict['User_Handle'] = i
        for j in range(0, cluster_count):
            user_dict["Cluster_"+str(j)] = sum(temp_df['Prob_'+str(j)])/post_count
        cluster_presence.append(user_dict)
    return cluster_presence

#Create Cluster Presence Dataframe
presence_list = cluster_presence(df)
df_presence = pd.DataFrame(presence_list)
df_presence = df_presence.fillna(0)
display(df_presence)

In [ ]:
#Define Cluster Count
cluster_count = 4

#Define Dataset
data_presence = df_presence.copy(deep=True)
del data_presence["User_Handle"]

#Implement K-Means Algortihm
model_kmeans_users = KMeans(n_clusters=cluster_count, random_state=9001)

#Fit Model and Predict
model_kmeans_users.fit(data_presence)
print("Representative User Vectors:\n\n",model_kmeans_users.cluster_centers_)

In [ ]:
#Helper Function to get images from target path 
def get_images_test(path):
    #Get all Images of Users in the List
    user_imgs = {}
    for j in glob.glob(path + '/*.jpg'):
        file_name = j.replace(path,'')[1:]
        img = io.imread(j)
        user_imgs[("Target",file_name)] = img
    return user_imgs 

In [ ]:
#Define target image folder path
target_path = "/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/Images/Test/"
target_dict = get_images_test(target_path)

#Convert image to feature vector and define columns
target_df = pd.DataFrame(convert(target_dict))
target_df.columns = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", 
                "G_MED", "B_Mean", "B_STD", "B_MED", "Canny", "ORB_X", "ORB_Y"]

#Define Target DF
data = target_df.copy(deep=True)
del data["User_Handle"]
del data["URL"]

#Make Prediction
y_pred = model_gaussian.predict_proba(data)

#Add prediction to dataframe     
for i in range(0,cluster_count+1):
    if i==cluster_count:
        label = "Prediction"
        target_df[label] = model_gaussian.predict(data)
        break
    label = "Prob_" + str(i)
    target_df[label] = y_pred[:,i]

display(target_df.head())

In [ ]:
#Helper function to generate distance dictionary
def get_dist_dict(df, k, model):
    final_dict = {}
    for i in range(0,len(df)):
        temp_file = df.iloc[i,1]
        temp_dist = []
        for j in range(0, k):
            temp_dist.append(np.linalg.norm(df.iloc[i,14:14+k].astype(float)-model.cluster_centers_[j]))
        final_dict[temp_file] = temp_dist
    return final_dict


final = get_dist_dict(target_df, cluster_count, model_kmeans_users)
print(final)

In [ ]:
from sklearn.externals import joblib
joblib.dump(model_kmeans_users, 'model_kmeans.pkl') 
del model_kmeans_users
model_kmeans_users = joblib.load('model_kmeans.pkl')
print(model_kmeans_users.cluster_centers_)

In [1]:
%%file ClusteringTrain.py

import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.externals import joblib

class Clustering:
    
    #Initialization
    def __init__(self, train_path, save_path):
        self.train_path = train_path
        self.save_path = save_path
        
    #Helper Function to get images from train path
    def get_train_images(self, user_list):
        list_users = user_list
        self.train_imgs = {}
        for i in list_users:
            temp_path = self.train_path + i
            for j in glob.glob(temp_path + '/*.jpg'):
                file_name = j.replace(temp_path,'')[1:]
                img = io.imread(j)
                self.train_imgs[(i,file_name)] = img 
        print("Number of images loaded:", len(self.train_imgs))
    
    #Helper function to convert image to d-dimension vector for each image and 
    #return dataframe of all images
    def convert_to_features(self, columns):
        features = []
        for i in self.train_imgs.items():
            r_mean, r_std, r_med = np.mean(i[1][:,:,0].ravel()), np.std(i[1][:,:,0].ravel()), np.median(i[1][:,:,0].ravel())
            g_mean, g_std, g_med  = np.mean(i[1][:,:,1].ravel()), np.std(i[1][:,:,1].ravel()), np.median(i[1][:,:,1].ravel())
            b_mean, b_std, b_med  = np.mean(i[1][:,:,2].ravel()), np.std(i[1][:,:,2].ravel()), np.median(i[1][:,:,2].ravel())
            canny = np.mean(np.ravel(cv2.Canny(cv2.cvtColor(i[1], cv2.COLOR_BGR2HSV),100,200,L2gradient = True)))
            try:
                orb = cv2.ORB_create(100)
                kp = orb.detect(i[1],None)
                kp, des = orb.compute(i[1], kp)
                orb_centers = list(KMeans(1).fit([i.pt for i in kp]).cluster_centers_)
                orbx1, orby1 = orb_centers[0][0]*255/np.shape(i[1])[0], orb_centers[0][1]*255/np.shape(i[1])[1]
            except ValueError:
                continue
            features.append(np.array([i[0][0],i[0][1], r_mean, r_std, r_med, g_mean, g_std, g_med, b_mean, b_std, b_med, canny, orbx1, orby1]))
        df = pd.DataFrame(features, columns = columns)
        return df
    
    def model_images_fit(self, df, k, extra_cols, rand_state):
        data = df.copy(deep=True)
        
        #Delete reference columns
        for i in extra_cols:
            del data[i]

        #Implement Gaussian Mixture Model Algortihm 
        model = GaussianMixture(n_components=k, random_state=9001)

        #Fit Model and Predict
        model.fit(data)
        y_pred = model.predict_proba(data)

        #Add prediction to dataframe and return 
        for i in range(0,k+1):
            if i==k:
                label = "Prediction"
                df[label] = model.predict(data)
                break
            label = "Prob_" + str(i)
            df[label] = y_pred[:,i]
        return df, model
    
    def model_users_fit(self, df, k, extra_cols, rand_state):
        data = df.copy(deep=True)
        
        #Delete reference columns
        for i in extra_cols:
            del data[i]

        #Implement K-Means Algortihm
        model = KMeans(n_clusters=k, random_state=rand_state)

        #Fit Model, Predict and Return
        model.fit(data)
        y_pred = model.predict(data)
        df['Prediction'] = model.labels_
        return df, model
    
    #Helper function to create folders for Image Clustering
    def save_clusters(self, df, label):
        self.save_path += label + "/"
        for i in self.train_imgs.items():
            temp_row = df[df["URL"]==i[0][1]]
            try:
                name, pred_folder = i[0][1], str(temp_row['Prediction'].values[0])
            except IndexError:
                continue
            temp_path = self.save_path + "Cluster" + pred_folder + "/"
            if not os.path.exists(temp_path):
                os.makedirs(temp_path)
            io.imsave(temp_path+name, i[1])
        print("All Images Saved.")
            
    #Helper function to obtain percentage of Cluster Presence
    def get_cluster_presence(self, df, k):
        cluster_presence = []
        for i in list(df['User_Handle'].unique()):
            user_dict = {}
            temp_df = df[df['User_Handle']==i]
            post_count  =  len(temp_df)
            user_dict['User_Handle'] = i
            for j in range(0, k):
                user_dict["Cluster_"+str(j)] = sum(temp_df['Prob_'+str(j)])/post_count
            cluster_presence.append(user_dict)
        df_presence = pd.DataFrame(cluster_presence)
        df_presence = df_presence.fillna(0)
        return df_presence
    
    #Helper function to save model 
    def save_model(self, model, path):
        joblib.dump(model, path) 
        print("Model Saved.")

Overwriting ClusteringTrain.py


In [2]:
%%file ClusteringTest.py

import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from skimage.viewer import ImageViewer
import cv2
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.externals import joblib

class Ranking:
    
    #Initialization
    def __init__(self, target_path):
        self.target_path = target_path

    #Helper Function to get images from target path 
    def get_images_target(self):
        #Get all Images of Users in the List
        self.target_imgs = {}
        for j in glob.glob(self.target_path + '/*.jpg'):
            file_name = j.replace(self.target_path,'')[:]
            img = io.imread(j)
            self.target_imgs[("Input/Target",file_name)] = img
        print("Number of images loaded:", len(self.target_imgs))
    
    #Helper function to load saved model
    def load_model(self, path):
        model = joblib.load(path)
        print ("Model loaded.")
        return model

    #Helper function to convert image to d-dimension vector for each image and 
    #return dataframe of all images
    def convert_to_features(self, columns):
        features = []
        for i in self.target_imgs.items():
            r_mean, r_std, r_med = np.mean(i[1][:,:,0].ravel()), np.std(i[1][:,:,0].ravel()), np.median(i[1][:,:,0].ravel())
            g_mean, g_std, g_med  = np.mean(i[1][:,:,1].ravel()), np.std(i[1][:,:,1].ravel()), np.median(i[1][:,:,1].ravel())
            b_mean, b_std, b_med  = np.mean(i[1][:,:,2].ravel()), np.std(i[1][:,:,2].ravel()), np.median(i[1][:,:,2].ravel())
            canny = np.mean(np.ravel(cv2.Canny(cv2.cvtColor(i[1], cv2.COLOR_BGR2HSV),100,200,L2gradient = True)))
            try:
                orb = cv2.ORB_create(100)
                kp = orb.detect(i[1],None)
                kp, des = orb.compute(i[1], kp)
                orb_centers = list(KMeans(1).fit([i.pt for i in kp]).cluster_centers_)
                orbx1, orby1 = orb_centers[0][0]*255/np.shape(i[1])[0], orb_centers[0][1]*255/np.shape(i[1])[1]
            except ValueError:
                continue
            features.append(np.array([i[0][0],i[0][1], r_mean, r_std, r_med, g_mean, g_std, g_med, b_mean, b_std, b_med, canny, orbx1, orby1]))
        df = pd.DataFrame(features, columns = columns)
        return df

    #Helper function to make prediction for target images using image model
    def predict(self, df, model, k, cluster_names, extra_cols):
        data = df.copy(deep=True)
        
        #Delete reference columns
        for i in extra_cols:
            del data[i]
            
        #Make Prediction
        y_pred = model.predict_proba(data)
        
        #Add prediction to dataframe and return 
        for i in range(0,k+1):
            if i==k:
                label = "Prediction"
                df[label] = model.predict(data)
                break
            label = cluster_names[i] + " (" + str(i) + ")"
            df[label] = y_pred[:,i]
        return df
    
    #Helper function to generate distance dictionary
    def get_result(self, df, k, model):
        final_dict = {}
        for i in range(0,len(df)):
            temp_file = df.iloc[i,1]
            temp_dist = []
            for j in range(0, k):
                temp_dist.append(np.linalg.norm(df.iloc[i,14:14+k].astype(float)-model.cluster_centers_[j]))
            final_dict[temp_file] = temp_dist
        return final_dict

Overwriting ClusteringTest.py


In [3]:
import ClusteringTrain

/anaconda3/lib/python3.6/site-packages/skimage/viewer/utils/core.py:10: UserWarning: Recommended matplotlib backend is `Agg` for full skimage.viewer functionality.
  warn("Recommended matplotlib backend is `Agg` for full "


In [4]:
#Define paths
train_path = "/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/Datasets/sample_users_media/"
dest_path = "/Users/kmotwani/Desktop/Me - Local/Education/Courses/Capstone Project/Clustering_"

#Create Object
obj_train = ClusteringTrain.Clustering(train_path, dest_path)

In [5]:
#Define User List
#user_list = ['ana_brandine', 'vicky_regouli', 'luismiguelpss', 'ilariabiagini', 
#              'emnegg', 'kerendhahn', 'agiorgina', 'roulamatta', 'jussbieber9827', 
#              'eremiaheidr', 'eunhuiheo', 'anastasiakaps', 'achaelilsone', 'orit_talbi',
#              'sorayaalassmi', 'altonolnlis', 'vaso1977', 'theunrealobserver', 'nsb.koc',
#              'vivpeng', 'amrynevillek', 'danalev7', 'irienyree', 'lilachturgeman', 
#              'emel_karakoc', 'thiswhomustbekept', 'j_f_lil', 'ulietteearneye', 
#              'gilanaz', 'sarrahdolly', 'alexchahine97', 'photographerarson', 
#              'angecanindo', 'fiona_smithson', 'chelsea_xu620']

user_list = ['sarrahdolly', 'alexchahine97', 'photographerarson', 'chelsea_xu620']

#Get Train Images
obj_train.get_train_images(user_list)

Number of images loaded: 887


In [6]:
#Get train dataframe
train_df_cols = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", 
                "G_MED", "B_Mean", "B_STD", "B_MED", "Canny", "ORB_X", "ORB_Y"]

train_df = obj_train.convert_to_features(train_df_cols)
display(train_df.head())

,User_Handle,URL,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny,ORB_X,ORB_Y
0,sarrahdolly,19367624_101179547177951_1390150557162799104_n...,60.54667718272289,42.19724092944082,46.0,115.95750487329434,32.60043756841384,116.0,154.99817379706576,59.84647423921173,183.0,10.985707140658665,164.9659097623957,111.04306045532226
1,sarrahdolly,11875280_1178056292210579_2092095415_n.jpg,184.04617877492876,35.35372396838029,195.0,161.45551994301994,68.04532270297828,189.0,171.04471866096867,57.410411921227386,193.0,40.13616452991453,168.73200391928353,91.83816484715389
2,sarrahdolly,10735461_1407215272901698_149677391_n.jpg,81.98706787109376,53.23458707440772,78.0,64.063896484375,43.261115626473355,56.0,67.79611328125,33.462023024518665,61.0,12.42938232421875,75.28755110353232,138.2118056935072
3,sarrahdolly,11850363_502039876616776_1969297340_n.jpg,154.0544828402367,59.84234337996732,163.0,140.04713372781066,66.63991410840953,146.0,140.46451360946745,70.77093845686376,142.0,37.09902958579882,79.52617734938403,106.16357121805042
4,sarrahdolly,25017868_725944580942040_944431130495418368_n.jpg,200.38099865768135,57.38725202538483,221.0,141.86143299364454,72.7586487745246,159.0,105.58023641244795,66.37926170123202,96.0,16.535112590401052,95.47079070813558,215.38616946326357


In [7]:
#Define cluster count for users and images 
k = 4

In [8]:
#Fit Model
train_df, model_images = obj_train.model_images_fit(train_df, k, ["User_Handle","URL"], 9001)
display(train_df.head())

,User_Handle,URL,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny,ORB_X,ORB_Y,Prob_0,Prob_1,Prob_2,Prob_3,Prediction
0,sarrahdolly,19367624_101179547177951_1390150557162799104_n...,60.54667718272289,42.19724092944082,46.0,115.95750487329434,32.60043756841384,116.0,154.99817379706576,59.84647423921173,183.0,10.985707140658665,164.9659097623957,111.04306045532226,1.338911e-27,5.819216e-25,0.999976,0.000024,2
1,sarrahdolly,11875280_1178056292210579_2092095415_n.jpg,184.04617877492876,35.35372396838029,195.0,161.45551994301994,68.04532270297828,189.0,171.04471866096867,57.410411921227386,193.0,40.13616452991453,168.73200391928353,91.83816484715389,2.422445e-29,5.413689e-11,0.999993,0.000007,2
2,sarrahdolly,10735461_1407215272901698_149677391_n.jpg,81.98706787109376,53.23458707440772,78.0,64.063896484375,43.261115626473355,56.0,67.79611328125,33.462023024518665,61.0,12.42938232421875,75.28755110353232,138.2118056935072,7.786541e-37,9.992929e-01,0.000512,0.000195,1
3,sarrahdolly,11850363_502039876616776_1969297340_n.jpg,154.0544828402367,59.84234337996732,163.0,140.04713372781066,66.63991410840953,146.0,140.46451360946745,70.77093845686376,142.0,37.09902958579882,79.52617734938403,106.16357121805042,9.569513e-05,4.251986e-02,0.949140,0.008244,2
4,sarrahdolly,25017868_725944580942040_944431130495418368_n.jpg,200.38099865768135,57.38725202538483,221.0,141.86143299364454,72.7586487745246,159.0,105.58023641244795,66.37926170123202,96.0,16.535112590401052,95.47079070813558,215.38616946326357,2.806121e-20,2.270893e-07,0.999870,0.000130,2


In [9]:
#Save Clusters to Local Directory
obj_train.save_clusters(train_df, "Users")

All Images Saved.


In [10]:
#Get each user's presence in each cluster 
presence_df = obj_train.get_cluster_presence(train_df, k)
display(presence_df.head())

,Cluster_0,Cluster_1,Cluster_2,Cluster_3,User_Handle
0,2.139363e-01,2.119507e-01,0.295228,2.788855e-01,sarrahdolly
1,2.625207e-05,6.191084e-01,0.290787,9.007835e-02,alexchahine97
2,3.333209e-01,1.368621e-05,0.000013,6.666524e-01,photographerarson
3,5.973326e-26,1.636503e-27,1.000000,2.175125e-07,chelsea_xu620


In [11]:
#Fit Presence Model
presence_df, model_users = obj_train.model_users_fit(presence_df, k, ["User_Handle"], 9001)
display(presence_df.head())

,Cluster_0,Cluster_1,Cluster_2,Cluster_3,User_Handle,Prediction
0,2.139363e-01,2.119507e-01,0.295228,2.788855e-01,sarrahdolly,3
1,2.625207e-05,6.191084e-01,0.290787,9.007835e-02,alexchahine97,0
2,3.333209e-01,1.368621e-05,0.000013,6.666524e-01,photographerarson,1
3,5.973326e-26,1.636503e-27,1.000000,2.175125e-07,chelsea_xu620,2


In [12]:
#Save Model
obj_train.save_model(model_images, "/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/model_images.plk")
obj_train.save_model(model_users, "/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/model_users.plk")

Model Saved.
Model Saved.


In [13]:
import ClusteringTest

In [14]:
#Define target path and create test object
target_path = "/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/Images/Test/"

obj_test = ClusteringTest.Ranking(target_path)

In [15]:
#Get Target Images
obj_test.get_images_target()

Number of images loaded: 2


In [16]:
#Get target dataframe
target_df_cols = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", 
                "G_MED", "B_Mean", "B_STD", "B_MED", "Canny", "ORB_X", "ORB_Y"]
target_df = obj_test.convert_to_features(target_df_cols)

In [17]:
#Load Model
model_images = obj_test.load_model("/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/model_images.plk")
model_users = obj_test.load_model("/Users/kmotwani/Dropbox/Harvard/Capstone_EmpSirenuse/model_users.plk")

Model loaded.
Model loaded.


In [18]:
#Get Prediction Dataframe
target_df = obj_test.predict(target_df, model_images, k, ["Philosophical/White","Nightlife/Black","Colorful","Random"], ["User_Handle","URL"])
display(target_df.head())

,User_Handle,URL,R_Mean,R_STD,R_MED,G_Mean,G_STD,G_MED,B_Mean,B_STD,B_MED,Canny,ORB_X,ORB_Y,Philosophical/White (0),Nightlife/Black (1),Colorful (2),Random (3),Prediction
0,Input/Target,test1.jpg,168.21228300623292,50.559814473532846,177.0,152.20649178385617,55.545941306964636,157.0,133.1401432030083,59.39653323057275,132.0,29.697457889043424,157.3104023982487,124.26272193696765,5.966905e-02,0.003429,0.935074,0.001828,2
1,Input/Target,12918414_1535355823435679_1454700275_n.jpg,69.02685501717532,64.38749628014699,47.0,101.4987613951645,70.35706785447088,92.0,125.40358287092086,56.03682228282825,122.0,46.59386766745937,111.95725933929968,133.13202668880595,1.990474e-42,0.996825,0.003119,0.000055,1


In [19]:
#Get Distance Dictionary
dist_dict = obj_test.get_result(target_df, k, model_users)
print("Distance to Cluster Dictionary:\n\n", dist_dict)

Distance to Cluster Dictionary:

 {'test1.jpg': [0.8975036955207851, 1.1795029253249076, 0.08826570922679301, 0.7439383335939207], '12918414_1535355823435679_1454700275_n.jpg': [0.48324614772849417, 1.2446274827103567, 1.4097627490039382, 0.9082236999348761]}
